# Scenario Discovery Demo: PRIM, PCA–PRIM, and CART

This interactive notebook demonstrates how three scenario discovery methods identify regions of interest in a synthetic 2D input space:

- **PRIM**: isolates axis-aligned boxes using iterative peeling
- **PCA-PRIM**: applies PRIM in PCA-rotated space to allow oblique cuts
- **CART**: uses decision trees to partition the space into class regions

Use the configuration panel to generate data, adjust parameters, and compare the methods side-by-side.

All output figures are saved to: [`../_data/scenario_methods_demo_outputs/`](../_data/scenario_methods_demo_outputs/)

### Slider Descriptions
The user interface provides adjustable sliders to control data generation and scenario discovery behavior. These parameters affect the shape, noise level, and algorithmic thresholds:

- **Num Dots**: Sets the total number of points in the unit square \([0, 1]^2\). Higher values improve resolution but increase runtime.

- **Corner X/Y (1–4)**: Sets the quadrilateral's four vertices, which determine the region. Adjusting them changes its rotation, skew or size. This can be useful for testing sensitivity of methods to shape orientation.

- **Frac Inside**: Sets the probability that points inside the ground truth shape are labeled as of interest (class 1). Lower values increase ambiguity within the region, introducing internal label uncertainty that makes it harder for methods to form compact, high-purity boxes.

- **Frac Outside**: Sets the probability that points outside the shape are labeled as of interest (class 1). Higher values increase external label ambiguity, forcing methods to differentiate relevant and irrelevant areas in the presence of scattered positive labels beyond the true region.

- **Peel Frac**: Sets how aggressively PRIM peels low-density regions. Larger values speed up peeling but may overshoot; smaller values offer many boxes to choose from, and a more detailed peeling trajectory.

- **PRIM Mass Min**: Sets the minimum fraction of data that a PRIM box must contain. Higher values make the process stop earlier.

- **CART Mass Min**: Sets the minimum mass per leaf in the CART tree. Smaller values allow higher complexity trees but may overfit; higher values yield lower complexity trees.

This block section loads necessary modules and configures the environment.

In [1]:
# === Imports and Setup ===
import os
import sys
sys.path.insert(0, os.path.abspath("..")) 

import numpy as np
import ipywidgets as widgets
from IPython.display import display

from notebook_helpers import update_plots, save_prim_plots, save_cart_plots

## Configuration Controls

This block sets up the sliders which define how the synthetic dataset is generated and how each algorithm behaves.

In [2]:
# Create input widgets
num_dots_slider = widgets.IntSlider(value=1700, min=100, max=2500, step=100, description="Num Dots")
default_quad = np.array([0.3, 0.9, 0.9, 0.7, 1.0, 0.4, 0.2, 0.55])
corner_labels = ["Corner 1 X", "Corner 1 Y", "Corner 2 X", "Corner 2 Y",
                 "Corner 3 X", "Corner 3 Y", "Corner 4 X", "Corner 4 Y"]
quad_sliders = [widgets.FloatSlider(value=val, min=0.0, max=1.0, step=0.05, description=lbl)
                for lbl, val in zip(corner_labels, default_quad)]
frac_inside_slider = widgets.FloatSlider(value=0.95, min=0.7, max=1.0, step=0.05, description="Frac Inside")
frac_outside_slider = widgets.FloatSlider(value=0.05, min=0.0, max=0.3, step=0.05, description="Frac Outside")
peel_frac_slider = widgets.FloatSlider(value=0.10, min=0.0, max=0.5, step=0.05, description="Peel Frac")
prim_mass_min_slider = widgets.FloatSlider(value=0.05, min=0.0, max=1.0, step=0.01, description="PRIM Mass Min")
cart_mass_min_slider = widgets.FloatSlider(value=0.05, min=0.0, max=1.0, step=0.01, description="CART Mass Min")
save_prim_button = widgets.Button(description="Save PRIM Plots")
save_cart_button = widgets.Button(description="Save CART Plots")

## Output Grid, Update Logic, and Save Button Setup

This block prepares the interactive infrastructure for the notebook interface. It defines the output layout, update behavior, and export functionality, but does **not yet render** anything visually.

In [3]:
# Create output areas
plot_outputs = [widgets.Output(layout=widgets.Layout(width="100%", height="300px"))
                for _ in range(9)]
table_output = widgets.Output(layout=widgets.Layout(width="100%", height="150px"))
grid = widgets.GridspecLayout(3, 3, width="12in", height="9in")
for i in range(3):
    for j in range(3):
        grid[i, j] = plot_outputs[i * 3 + j]
        
def on_update(_):
    update_plots(
        quad_sliders,
        num_dots_slider.value,
        frac_inside_slider.value,
        frac_outside_slider.value,
        peel_frac_slider.value,
        prim_mass_min_slider.value,
        cart_mass_min_slider.value,
        plot_outputs,
        table_output,
    )
    
# Attach observers
num_dots_slider.observe(on_update, names="value")
frac_inside_slider.observe(on_update, names="value")
frac_outside_slider.observe(on_update, names="value")
peel_frac_slider.observe(on_update, names="value")
prim_mass_min_slider.observe(on_update, names="value")
cart_mass_min_slider.observe(on_update, names="value")
for slider in quad_sliders:
    slider.observe(on_update, names="value")

# Initial update
on_update(None)

# Set up the save buttons to export the plots
save_prim_button.on_click(lambda b: save_prim_plots(
    quad_sliders,
    num_dots_slider.value,
    frac_inside_slider.value,
    frac_outside_slider.value,
    peel_frac_slider.value,
    prim_mass_min_slider.value,
    cart_mass_min_slider.value,
))
save_cart_button.on_click(lambda b: save_cart_plots(
    quad_sliders,
    num_dots_slider.value,
    frac_inside_slider.value,
    frac_outside_slider.value,
    peel_frac_slider.value,
    prim_mass_min_slider.value,
    cart_mass_min_slider.value
))

## Displaying the Interactive Interface

This cell lays out and renders the complete user interface:

- All **input sliders** are grouped into vertical and horizontal boxes (`VBox`, `HBox`) for clean visual alignment.
- The **output grid** of plots and the **CART results table** are placed below the controls.
- The `display(...)` function renders everything in the notebook for user interaction.

From here, you can:
- Adjust any slider to dynamically regenerate the plots.
- Click "Save PRIM Plots" or "Save CART Plots" to export the current results.

All exported figures are saved under: `_data/scenario_methods_demo_outputs/`

In [4]:
input_widgets = widgets.VBox([
    num_dots_slider,
    widgets.HBox([frac_inside_slider, frac_outside_slider, peel_frac_slider]),
    widgets.HBox(quad_sliders[:4]),
    widgets.HBox(quad_sliders[4:]),
    widgets.HBox([prim_mass_min_slider, cart_mass_min_slider]),
    save_prim_button,
    save_cart_button,
])
display(input_widgets, grid, table_output)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001', height='300px', width='100%')), Output(la…

Output(layout=Layout(height='585px', width='100%'))